In [ ]:
#
#
#
# 이거는 테스트용!!!!!!!!!!!!!!!!!!
# 크롤러를 만들기 전 필요한 도구 임포트
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import asyncio
import requests
import json
import traceback
import aiofiles

news_code = 101
stop_count = 500

# 형태소 분석 시작
# pip install konlpy 터미널에 입력 후 시작
# Mecab 이란 것도 있긴 하나 안됨...시도도 하지 마시오...(제발)
# 데이터 정제
from konlpy.tag import Okt
tokenizer = Okt()

from konlpy.tag import Kkma
kkma = Kkma()

stopwords = ['멘트', '앵커', '기자', 'KBS', 'MBC', 'SBS', '와', '과', '가', '의', '이', '은', '등', '을', '이다', '했다', '구독', 
            'http', 'https',':', '···', '…', '’', '‘', 'ⓒ', "!", "'", '"', '는', '를', '=', '[', ']', '(', ')', '.', ',','·', '으로', '로', '에', '도', '고']
containwords = ['할인', '상승', '하락', '물가', '가격', '비싸', '금액', '제사', '마트', '장바구니']
specific_words = ['쌀', '찹쌀', '콩', '팥', '녹두', '메밀', '고구마', '감자', '귀리', '혼합곡', '보리', '수수', '율무', '기장', '배추', '양배추', '알배기배추', '브로콜리', '시금치', '상추', '얼갈이 배추',
                '갓', '수박', '참외', '오이', '호박', '토마토', '딸기', '무', '당근', '열무', '건고추', '풋고추', '붉은고추', '고추', '피마늘', '깐마늘', '마늘', '양파', '파', '생강', '고춧가루', '가지',
                '미나리', '깻잎', '부추', '피망', '파프리카', '멜론', '브로콜리', '양상추', '상추', '청경채', '케일', '콩나물', '방울토마토', '연근', '우엉', '절임배추', '참깨', '들깨', '땅콩', '느타리버섯',
                '팽이버섯', '새송이버섯', '버섯', '호두', '아몬드', '양송이버섯', '표고버섯', '견과류', '사과', '배', '복숭아', '포도', '감귤', '귤', '단감', '감', '바나나', '참다래', '파인애플', '오렌지',
                '자몽', '레몬', '체리', '건포도', '건블루베리', '망고', '아보카도', '블루베리',
                '수확', '김장', '채소', '과일', '농가', '식재료', '수확', '작황', '농산물','농작물', '생육', '곡물']

# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
# async def preprocessing(csv_path, doing_date):
#     # 저장한 CSV 파일 다시 읽어오기
#     #csv_path = str(news_code) + "-" + doing_date + ".csv"
#     print(doing_date + " 데이터 처리 시작")
#     save_folder = "done-" + doing_date[:6]
#     df = pd.read_table(csv_path, sep=',')
#     data = df['news']
#     text_data = []

#     specific = 0
#     data_count = 0

#     # 뉴스 기사 하나 단위
#     for sentence in data:
#         final_data = []
#         data_count += 1
#         temp_data = []
#         exist_data = []
#         specific_data = []
#         try:
#             #- 토큰화
#             s = kkma.sentences(sentence)    # 문장 단위로 자른 데이터
#             for one in s:
#                 temp_data = tokenizer.morphs(one) 
#                 #- 불용어 제거
#                 temp_data = [word for word in temp_data if not word in stopwords]
#                 exist_data= [word for word in temp_data if word in containwords]
#                 specific_data = [word for word in temp_data if word in specific_words]
#                 if len(exist_data) >= 4 or len(specific_data) >=1:
#                     final_data.append(one)
#             final_data = list(map(''.join, final_data))
#             if len(final_data) > 0 :
#                 text_data.append(final_data)
#         except:
#             continue
#         if data_count >= stop_count:
#             print(doing_date + " 뉴스 기사" + str(stop_count) + " 개 완료.")
#             data_count = 0
#             #await asyncio.sleep(0.1)
#         #print("가능한 단어는 = " + str(len(exist_data)))
#     text_data = list(map(' '.join, text_data))
#     filtered = pd.DataFrame({'news': text_data})
#     # 폴더가 없다면 년월에 해당하는 폴더 생성
#     try:
#         if not os.path.exists(save_folder):
#             os.makedirs(save_folder)
#     except:
#         print(doing_date)
#     # csv 파일로 저장
#     now_path = save_folder + "/" + doing_date + ".json"
#     filtered.to_json(now_path, orient='values', force_ascii=False, indent=4)

#     if os.path.exists(now_path):
#         print('{} File Saved!'.format(now_path))

async def preprocessing(csv_list: list, date_list: list):
    # 저장한 CSV 파일 다시 읽어오기
    tasks = []
    for csv_path, doing_date in zip(csv_list, date_list) :
        task = news_filtering(csv_path, doing_date)
        tasks.append(task)

    
    print("task 적재 완료")
    await asyncio.gather(*tasks)
    # save_folder = "done-" + doing_date[:6]
    # df = pd.read_table(csv_path, sep=',')
    # data = df['news']
    # text_data = []

    # specific = 0
    # data_count = 0

    # # 뉴스 기사 하나 단위
    # for sentence in data:
    #     final_data = []
    #     data_count += 1
    #     temp_data = []
    #     exist_data = []
    #     specific_data = []
    #     try:
    #         #- 토큰화
    #         s = kkma.sentences(sentence)    # 문장 단위로 자른 데이터
    #         for one in s:
    #             temp_data = tokenizer.morphs(one) 
    #             #- 불용어 제거
    #             temp_data = [word for word in temp_data if not word in stopwords]
    #             exist_data= [word for word in temp_data if word in containwords]
    #             specific_data = [word for word in temp_data if word in specific_words]
    #             if len(exist_data) >= 4 or len(specific_data) >=1:
    #                 final_data.append(one)
    #         final_data = list(map(''.join, final_data))
    #         if len(final_data) > 0 :
    #             text_data.append(final_data)
    #     except:
    #         continue
    #     if data_count >= stop_count:
    #         print(doing_date + " 뉴스 기사" + str(stop_count) + " 개 완료.")
    #         data_count = 0
    #         #await asyncio.sleep(0.1)
    #     #print("가능한 단어는 = " + str(len(exist_data)))
    # text_data = list(map(' '.join, text_data))
    # filtered = pd.DataFrame({'news': text_data})
    # # 폴더가 없다면 년월에 해당하는 폴더 생성
    # try:
    #     if not os.path.exists(save_folder):
    #         os.makedirs(save_folder)
    # except:
    #     print(doing_date)
    # # csv 파일로 저장
    # now_path = save_folder + "/" + doing_date + ".json"
    # filtered.to_json(now_path, orient='values', force_ascii=False, indent=4)

    # if os.path.exists(now_path):
    #     print('{} File Saved!'.format(now_path))


# 생각 생각 생각을 다시 해봅시다...
# 뉴스 기사가 하루에 대충 4000개니까
# 하루에 500개씩 단위를 끊어서 해 주는 것이 맞다...
# 이게 맞는지는 모르겠지만
# url처럼 500개씩 받아서 받으면 형태소 처리해주고 append...
# 해보고 안되면 도르마무 
# 라고 했는데 애초에... 거 뭐냐 데이터를 따로 받는게 안되는군...
async def news_filtering(csv_path, doing_date) :
    print(doing_date + "뉴스 데이터 필터링 시작")
    save_folder = "done-" + doing_date[:6]
    async with aiofiles.open(csv_path, encoding='utf-8') as open_csv:
        df = pd.read_table(open_csv, sep=',')
    data = df['news']
    text_data = []

    specific = 0
    data_count = 0

    # 뉴스 기사 하나 단위
    for sentence in data:
        final_data = []
        data_count += 1
        temp_data = []
        exist_data = []
        specific_data = []
        try:
            #- 토큰화
            s = kkma.sentences(sentence)    # 문장 단위로 자른 데이터
            for one in s:
                temp_data = tokenizer.morphs(one) 
                #- 불용어 제거
                temp_data = [word for word in temp_data if not word in stopwords]
                exist_data= [word for word in temp_data if word in containwords]
                specific_data = [word for word in temp_data if word in specific_words]
                if len(exist_data) >= 4 and len(specific_data) >=1 or len(specific_data)>=2:
                    final_data.append(one)
            final_data = list(map(''.join, final_data))
            if len(final_data) > 0 :
                text_data.append(final_data)
        except:
            continue
        if data_count >= stop_count:
            print(doing_date + " 뉴스 기사" + str(stop_count) + " 개 완료.")
            data_count = 0
            #await asyncio.sleep(0.1)
        #print("가능한 단어는 = " + str(len(exist_data)))
    text_data = list(map(' '.join, text_data))
    filtered = pd.DataFrame({'news': text_data})
    # 폴더가 없다면 년월에 해당하는 폴더 생성
    try:
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
    except:
        print(doing_date)
    # csv 파일로 저장
    now_path = save_folder + "/" + doing_date + ".json"
    filtered.to_json(now_path, orient='values', force_ascii=False, indent=4)

    if os.path.exists(now_path):
        print('{} File Saved!'.format(now_path))



# 5년치 데이터 형태소 분석
async def main():
    start_date = '20190101'
    end_date = '20190110'
    doing_date = start_date
    tasks = []
    count_days = 0
    csv_list=[]
    date_list=[]
    while(doing_date <= end_date):
        # 저장한 CSV 파일 다시 읽어오기
        csv_path = doing_date[:6] + "/" + str(news_code) + "-" + str(doing_date) + ".csv"
        # print(csv_path)
        try:
            if os.path.exists(csv_path):
                print(csv_path)
                csv_list.append(csv_path)
                date_list.append(doing_date)
                #task = preprocessing(csv_path, doing_date)
                count_days += 1
                #tasks.append(task)

            # 3일 단위로 데이터 가져오기
            if count_days >= 10 or doing_date == end_date:
                await preprocessing(csv_list, date_list)
                #tasks = []
                csv_list =[]
                date_list = []
                count_days = 0
                await asyncio.sleep(0.5)  # 부하로 인한 차단을 막기 위해 일시정지 후 재시작
        except:
            print("error발생, 넘어갑니다.")
            error_msg = traceback.format_exc()
            print(error_msg)
            #continue

        # 날짜 +1 해주기
        doing = datetime.datetime.strptime(doing_date, '%Y%m%d')
        doing = doing + datetime.timedelta(days = 1)
        doing_date = datetime.datetime.strftime(doing, "%Y%m%d")

await main()

